In [63]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [64]:
import json
import json_lines
import preprocess_temp as P
import model.parsers as M

### Load Data

In [65]:
directory = './conala-corpus/'
train_file = directory + 'train.json'
test_file = directory + 'test.json'

with open(train_file) as f:
    train_data = json.load(f)
    
with open(test_file) as f:
    test_data = json.load(f)

In [67]:
mine_file = directory + 'mined.jsonl'
mine_data = []
with open(mine_file) as f:
    mine_data = [line for line in json_lines.reader(f)]

### Let's preprocess the data. Everything is in Preprocess.py
### Adding mined data

In [68]:
mine_data[0]

{'id': '34705205_34705233_0',
 'intent': 'Sort a nested list by two elements',
 'parent_answer_post_id': 34705233,
 'prob': 0.8690001442846342,
 'question_id': 34705205,
 'snippet': 'sorted(l, key=lambda x: (-int(x[1]), x[0]))'}

In [69]:
# intent processing includes lowercase, remove punctuation'?'
train_intent, train_codes = P.process_data(train_data)
test_intent, test_codes = P.process_data(test_data)

In [70]:
mine_intent, mine_codes = P.process_data(mine_data, mine=True)

In [71]:
# this class is used for code2actions and actions2code
ast_action = P.Ast_Action()

In [75]:
train_actions = []

for code in train_codes:
    train_actions.append(ast_action.code2actions(code))

In [77]:
word_lst = P.vocab_list(train_intent, cut_freq=2)
act_lst, token_lst = P.action_list(train_actions, cut_freq=0)

In [78]:
word2num = dict(zip(word_lst, range(0,len(word_lst))))
act2num = dict(zip(act_lst, range(0,len(act_lst))))
token2num = dict(zip(token_lst, range(0,len(token_lst))))

In [79]:
train_loader = P.get_train_loader(train_intent, train_actions, word2num, act2num, token2num, batch_size=48)

In [108]:
test_loader = P.get_test_loader(test_intent, word2num, batch_size=1)

In [81]:
action_index_copy = act2num[P.GenTokenAction('copy')]
action_index_gen = act2num[P.GenTokenAction('token')]

### Model

In [82]:
import torch
import time

In [83]:
from collections import namedtuple
hyperParamMap = {
    #### General configuration ####
    'cuda': True,      # Use gpu
    'mode': 'train',   # train or test

    #### Embedding sizes ####
    'embed_size': 128,         # Size of word embeddings
    'action_embed_size': 128,  # Size of ApplyRule/GenToken action embeddings
    'field_embed_size': 64,    # Embedding size of ASDL fields
    'type_embed_size': 64,     # Embeddings ASDL types

    #### Decoding sizes ####
    'hidden_size': 256,        # Size of LSTM hidden states

    #### training schedule details ####
    'valid_metric': 'acc',                # Metric used for validation
    'valid_every_epoch': 1,               # Perform validation every x epoch
    'log_every': 10,                      # Log training statistics every n iterations
    'save_to': 'model',                   # Save trained model to
    'clip_grad': 5.,                      # Clip gradients
    'max_epoch': 10,                      # Maximum number of training epoches
    'optimizer': 'Adam',                  # optimizer
    'lr': 0.001,                          # Learning rate
    'lr_decay': 0.5,                      # decay learning rate if the validation performance drops
    'verbose': False,                     # Verbose mode

    #### decoding/validation/testing ####
    'load_model': None,                   # Load a pre-trained model
    'beam_size': 1,                       # Beam size for beam search
    'decode_max_time_step': 100,          # Maximum number of time steps used in decoding and sampling
    'sample_size': 5,                     # Sample size
    'test_file': '',                      # Path to the test file
    'save_decode_to': None,               # Save decoding results to file
}

HyperParams = namedtuple('HyperParams', list(hyperParamMap.keys()), verbose=False)
hyperParams = HyperParams(**hyperParamMap)

In [84]:
model = M.Model(hyperParams, action_size=len(act_lst), token_size=len(token_lst), word_size=len(word_lst), 
                      action_index_copy=action_index_copy, action_index_gen=action_index_gen)

In [85]:
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
lossFunc = torch.nn.CrossEntropyLoss()

In [103]:
model.train()
epoch_begin = time.time()
for e in range(20):
    for batch_ind, x in enumerate(train_loader):
        optimizer.zero_grad()

        (action_logits, action_labels), (copy_logits, copy_labels), (token_logits, token_labels) = model(x)

        loss1 = lossFunc(action_logits, action_labels)
        loss2 = torch.DoubleTensor([0.0])
        if len(copy_logits) > 0:
            loss2 = lossFunc(copy_logits, copy_labels)
        loss3 = torch.DoubleTensor([0.0])
        if len(token_logits) > 0:
            loss3 = lossFunc(token_logits, token_labels)

        total_loss = loss1 + loss2.double() + loss3.double()
        total_loss.backward()

        # clip gradient
        if hyperParams.clip_grad > 0.:
            grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), hyperParams.clip_grad)

        optimizer.step()

        if batch_ind % hyperParams.log_every == hyperParams.log_every - 1:
            print('--------------------------epoch {} batch {}-----------------------------'.format(e, batch_ind))
            print("Action loss: {}".format(loss1.data))
            print("Copy loss: {}".format(loss2.data))
            print("Token loss: {}".format(loss3.data))
            report_loss = report_examples = 0.

    print('epoch elapsed %ds' % (time.time() - epoch_begin))

--------------------------epoch 0 batch 9-----------------------------
Action loss: 0.2744177109969296
Copy loss: 1.516951560974121
Token loss: 1.560115098953247


--------------------------epoch 0 batch 19-----------------------------
Action loss: 0.36408076626704355
Copy loss: 1.7325929403305054
Token loss: 1.5815476179122925


--------------------------epoch 0 batch 29-----------------------------
Action loss: 0.3117624012652552
Copy loss: 1.6275923252105713
Token loss: 1.5526243448257446


--------------------------epoch 0 batch 39-----------------------------
Action loss: 0.2580236762237825
Copy loss: 1.7134811878204346
Token loss: 1.5814095735549927


--------------------------epoch 0 batch 49-----------------------------
Action loss: 0.33341637713412564
Copy loss: 1.6383650302886963
Token loss: 1.7174134254455566
epoch elapsed 100s


...

--------------------------epoch 7 batch 19-----------------------------
Action loss: 0.27155659665437537
Copy loss: 1.339073896408081
Token loss: 0.7629060745239258


--------------------------epoch 7 batch 29-----------------------------
Action loss: 0.31005029744395624
Copy loss: 1.6809741258621216
Token loss: 0.7759092450141907


--------------------------epoch 7 batch 39-----------------------------
Action loss: 0.3321734084995204
Copy loss: 1.6711399555206299
Token loss: 0.8589382767677307


--------------------------epoch 7 batch 49-----------------------------
Action loss: 0.36996231742292507
Copy loss: 1.6980434656143188
Token loss: 0.8561928272247314
epoch elapsed 797s


KeyboardInterrupt: 

In [117]:
torch.save((model).state_dict(), 'Parameters/cut_2_0_v1.t7')

In [53]:
model.load_state_dict(torch.load('Parameters/frist.t7'))

In [118]:
sample_sent, sample_sent_txt = next(iter(test_loader))
print(sample_sent)
print(sample_sent_txt)
sample_hypothesis = model.parse(sample_sent, sample_sent_txt, act_lst, token_lst, ast_action)
ast_action.actions2code(sample_hypothesis.actions)

[tensor([ 11,   4,   2,  14, 119,  12,   4,   2,   7])]
[['convert', 'list', 'of', 'string', 'numbers', 'into', 'list', 'of', 'integers']]
[ApplyRule[stmt -> Expr(expr value)]]
[ApplyRule[stmt -> Expr(expr value)], ApplyRule[expr -> Call(expr func, expr* args, keyword* keywords)]]
[ApplyRule[stmt -> Expr(expr value)], ApplyRule[expr -> Call(expr func, expr* args, keyword* keywords)], ApplyRule[expr -> Name(identifier id)]]
[ApplyRule[stmt -> Expr(expr value)], ApplyRule[expr -> Call(expr func, expr* args, keyword* keywords)], ApplyRule[expr -> Name(identifier id)], GenToken[map]]
[ApplyRule[stmt -> Expr(expr value)], ApplyRule[expr -> Call(expr func, expr* args, keyword* keywords)], ApplyRule[expr -> Name(identifier id)], GenToken[map], ApplyRule[expr -> Call(expr func, expr* args, keyword* keywords)]]
[ApplyRule[stmt -> Expr(expr value)], ApplyRule[expr -> Call(expr func, expr* args, keyword* keywords)], ApplyRule[expr -> Name(identifier id)], GenToken[map], ApplyRule[expr -> Call(exp

"map(map(',', '', ''))"

In [149]:
my_t_train_loader = P.get_test_loader(train_intent, word2num, batch_size=1)
sample_sent, sample_sent_txt = next(iter(my_t_train_loader))
print(sample_sent)
print(sample_sent_txt)
sample_hypothesis = model.parse(sample_sent, sample_sent_txt, act_lst, token_lst, ast_action)
ast_action.actions2code(sample_hypothesis.actions)

[tensor([991, 557, 156, 418,  51, 111, 992, 189,  25,  60])]
[['Can', 'I', 'sort', 'text', 'by', 'its', 'numeric', 'value', 'in', 'Python']]
copy_logits size torch.Size([10])
copy_ind 2


"mylist.sort('test.csv', **'b')"

In [161]:
print(sample_hypothesis.actions)
ast_action.actions2code(sample_hypothesis.actions)

[ApplyRule[stmt -> Expr(expr value)], ApplyRule[expr -> Call(expr func, expr* args, keyword* keywords)], ApplyRule[expr -> Name(identifier id)], GenToken[any], ApplyRule[expr -> GeneratorExp(expr elt, comprehension* generators)], ApplyRule[expr -> Compare(expr left, cmpop* ops, expr* comparators)], ApplyRule[expr -> Call(expr func, expr* args, keyword* keywords)], ApplyRule[expr -> Name(identifier id)], GenToken[isinstance], ApplyRule[expr -> Name(identifier id)], GenToken[x], Reduce, Reduce, Reduce, Reduce, ApplyRule[comprehension -> comprehension(expr target, expr iter, expr* ifs)], ApplyRule[expr -> Name(identifier id)], GenToken[i], ApplyRule[expr -> Call(expr func, expr* args, keyword* keywords)], ApplyRule[expr -> Name(identifier id)], GenToken[range], ApplyRule[expr -> Num(object n)], GenToken[100], ApplyRule[expr -> Num(object n)], GenToken[9], Reduce, Reduce, Reduce, Reduce, Reduce, Reduce]
Expr(value=Call(func=Name(id='any'), args=[GeneratorExp(elt=Compare(left=Call(func=Name

IndexError: list index out of range

In [162]:
model.eval()
test_loader = P.get_test_loader(test_intent, word2num, batch_size=1)
testFile = open('./testl_predict.txt','w')
for (sample_sent, sample_sent_txt) in test_loader:
    sample_hypothesis = model.parse(sample_sent, sample_sent_txt, act_lst, token_lst, ast_action)
    try:
        code = ast_action.actions2code(sample_hypothesis.actions)
    except:
        code = ' '
    testFile.write(code+'\n')
testFile.close()

Expr(value=Call(func=Attribute(value=Name(id='os'), attr='system'), args=[Str(s='taskkill /im')], keywords=[]))
Expr(value=Call(func=Attribute(value=Name(id='os'), attr='savefig'), args=[Str(s='ulimit /im')], keywords=[]))
Expr(value=Call(func=Name(id='any'), args=[GeneratorExp(elt=Compare(left=Call(func=Name(id='isinstance'), args=[Name(id='x')], keywords=[]), ops=[], comparators=[]), generators=[comprehension(target=Name(id='i'), iter=Call(func=Name(id='range'), args=[Num(n=100), Num(n=9)], keywords=[]), ifs=[])])], keywords=[]))
Expr(value=Call(func=Attribute(value=Name(id='re'), attr='findall'), args=[Str(s='//*[contains(text(),'), Str(s='"\\1"')], keywords=[]))
Expr(value=Call(func=Attribute(value=Str(s=''), attr='decode'), args=[Str(s='Hello')], keywords=[]))
Expr(value=Call(func=Attribute(value=Name(id='df'), attr='apply'), args=[Str(s='myfile.csv')], keywords=[]))
Expr(value=Call(func=Attribute(value=Name(id='df'), attr='apply'), args=[Str(s='myfile.csv')], keywords=[]))


Expr(value=Call(func=Attribute(value=Name(id='plt'), attr='system'), args=[Str(s='script2.py')], keywords=[keyword(arg='$1', value=Str(s='./uireplace.pl'))]))
copy_logits size torch.Size([8])
copy_ind 0
Expr(value=Call(func=Name(id='concatenate'), args=[Call(func=Attribute(value=Str(s=''), attr='format'), args=[GeneratorExp(elt=Call(func=Name(id='str'), args=[Name(id='a')], keywords=[]), generators=[])], keywords=[])], keywords=[]))
Expr(value=Call(func=Attribute(value=Name(id='np'), attr='einsum'), args=[List(elts=[Str(s='Peak')])], keywords=[]))
Expr(value=Call(func=Attribute(value=Name(id='ax'), attr='set_xlabel'), args=[Str(s='TASKKILL screen.png')], keywords=[]))
copy_logits size torch.Size([9])
copy_ind 2
Expr(value=Call(func=Attribute(value=Attribute(value=Name(id='datetime'), attr='stdout'), attr='combine'), args=[Str(s='java')], keywords=[]))


Expr(value=Call(func=Attribute(value=Name(id='re'), attr='findall'), args=[Str(s='file'), Str(s='file.gz')], keywords=[]))
Expr(value=Call(func=Attribute(value=Name(id='re'), attr='findall'), args=[Str(s='My'), Str(s='Current'), Str(s='r')], keywords=[]))
copy_logits size torch.Size([9])
copy_ind 0
Expr(value=Call(func=Attribute(value=Name(id='a'), attr='concatenate'), args=[Str(s='a')], keywords=[]))
Expr(value=Call(func=Attribute(value=Name(id='re'), attr='match'), args=[Str(s='(?=(a.*?a))')], keywords=[keyword(arg='bbox_inches', value=Call(func=Attribute(value=Name(id='re'), attr='focus_get'), args=[], keywords=[]))]))
Expr(value=Subscript(value=Name(id='df'), slice=Index(value=Str(s='Test'))))
Expr(value=Call(func=Attribute(value=Name(id='ax'), attr='set_xlabel'), args=[Str(s='Temperature screen.png')], keywords=[]))


Expr(value=Call(func=Attribute(value=Str(s=''), attr='join'), args=[Name(id='s')], keywords=[]))
Expr(value=Call(func=Attribute(value=Name(id='os'), attr='system'), args=[Str(s='tcsh your_own_script /bin/bash "echo')], keywords=[]))
Expr(value=Call(func=Attribute(value=Str(s='0x{0:08X}'), attr='format'), args=[Str(s='.txt')], keywords=[]))
Expr(value=Call(func=Attribute(value=Name(id='struct'), attr='findall'), args=[Str(s='s'), Str(s='s')], keywords=[]))
Expr(value=Call(func=Attribute(value=Name(id='re'), attr='search'), args=[Str(s='[a-zA-Z]'), Str(s='')], keywords=[]))
Expr(value=Call(func=Attribute(value=Name(id='ax'), attr='get'), args=[Str(s='log')], keywords=[]))
Expr(value=Call(func=Attribute(value=Name(id='logging'), attr='findall'), args=[Str(s='file')], keywords=[]))
Expr(value=Call(func=Attribute(value=Name(id='subprocess'), attr='call'), args=[Str(s='append')], keywords=[]))


Expr(value=Call(func=Attribute(value=Name(id='requests'), attr='get'), args=[Str(s='Temperature (℃)')], keywords=[keyword(arg='headers', value=Dict(keys=[Str(s='pwd')], values=[]))]))
Expr(value=Call(func=Attribute(value=Name(id='requests'), attr='get'), args=[Str(s='Temperature (℃)')], keywords=[keyword(arg='headers', value=Dict(keys=[Str(s='pwd')], values=[]))]))
Expr(value=Call(func=Attribute(value=Name(id='requests'), attr='get'), args=[Str(s='log screen.png')], keywords=[]))
Expr(value=Call(func=Attribute(value=Name(id='requests'), attr='get'), args=[Str(s='log screen.png')], keywords=[]))
Expr(value=Call(func=Attribute(value=Name(id='requests'), attr='get'), args=[Str(s='Temperature (℃)')], keywords=[keyword(arg='headers', value=Dict(keys=[Str(s='pwd')], values=[]))]))
Expr(value=Call(func=Attribute(value=Name(id='ax'), attr='savefig'), args=[Str(s='awk document.body.scrollHeight);')], keywords=[keyword(arg='auth', value=Str(s='%b'))]))
Expr(value=DictComp(key=Name(id='k'), valu

...

Expr(value=Call(func=Attribute(value=Name(id='np'), attr='sort'), args=[List(elts=[Str(s='Cat1'), Str(s='City'), Str(s='City')])], keywords=[]))
Expr(value=ListComp(elt=Call(func=Name(id='sorted'), args=[Name(id='x')], keywords=[]), generators=[comprehension(target=Tuple(elts=[Name(id='x'), Name(id='v')]), iter=Call(func=Name(id='zip'), args=[Name(id='L')], keywords=[]), ifs=[])]))
Expr(value=Call(func=Attribute(value=Name(id='soup'), attr='findAll'), args=[Str(s='SELECT')], keywords=[keyword(arg='href', value=Str(s='name'))]))
Expr(value=Call(func=Attribute(value=Name(id='df'), attr='to_csv'), args=[Str(s='c:\\data\\t.csv')], keywords=[keyword(arg='sep', value=Str(s=';'))]))
Expr(value=Call(func=Attribute(value=Name(id='s'), attr='strip'), args=[], keywords=[]))
Expr(value=ListComp(elt=Call(func=Name(id='sum'), args=[Name(id='sublist')], keywords=[]), generators=[comprehension(target=Name(id='sublist'), iter=Name(id='sublist'), ifs=[])]))
Expr(value=Call(func=Attribute(value=Name(id='

Expr(value=Call(func=Attribute(value=Name(id='np'), attr='load'), args=[Str(s='monkey')], keywords=[keyword(arg='nargs', value=Str(s='\r'))]))
Expr(value=Call(func=Attribute(value=Name(id='np'), attr='load'), args=[Str(s='monkey')], keywords=[keyword(arg='nargs', value=Str(s='\r'))]))
Expr(value=Call(func=Attribute(value=Name(id='np'), attr='load'), args=[Str(s='monkey')], keywords=[keyword(arg='nargs', value=Str(s='\r'))]))
Expr(value=Call(func=Attribute(value=Name(id='np'), attr='load'), args=[Str(s='monkey')], keywords=[keyword(arg='nargs', value=Str(s='\r'))]))
Expr(value=Call(func=Attribute(value=Name(id='plt'), attr='info'), args=[Str(s='Aug')], keywords=[]))
Expr(value=Call(func=Attribute(value=Name(id='plt'), attr='info'), args=[Str(s='Aug')], keywords=[]))
Expr(value=ListComp(elt=Subscript(value=Name(id='x'), slice=Index(value=Num(n=0))), generators=[comprehension(target=Name(id='x'), iter=Name(id='lst'), ifs=[])]))
Expr(value=ListComp(elt=Subscript(value=Name(id='sublist'), s

Expr(value=Call(func=Name(id='list'), args=[Call(func=Name(id='list'), args=[Call(func=Attribute(value=Name(id='dictionary'), attr='items'), args=[], keywords=[])], keywords=[])], keywords=[]))
Expr(value=Call(func=Attribute(value=Str(s='  join join join join join                                 '), attr='</primitive>'), args=[Str(s=' '), Str(s=' '), Str(s=' '), Str(s=' '), Str(s=' '), Str(s=' ')], keywords=[]))
Expr(value=Call(func=Attribute(value=Name(id='np'), attr='findall'), args=[Str(s='a')], keywords=[]))
copy_logits size torch.Size([14])
copy_ind 0
Expr(value=Call(func=Attribute(value=Name(id='users'), attr='Sort'), args=[], keywords=[keyword(arg='key', value=Lambda(args=arguments(args=[arg(arg='x', annotation=None)], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=Call(func=Attribute(value=Name(id='order'), attr='index'), args=[Num(n=1)], keywords=[])))]))
Expr(value=Call(func=Attribute(value=Name(id='np'), attr='mean'), args=[Str(s='c:/~/trainSetRel

Expr(value=ListComp(elt=Name(id='x'), generators=[comprehension(target=Name(id='sublist'), iter=Name(id='sublist'), ifs=[Compare(left=Str(s='ar'), ops=[In()], comparators=[Name(id='sublist')])])]))
Expr(value=Call(func=Attribute(value=Subscript(value=Name(id='df'), slice=Index(value=Str(s='Test'))), attr='apply'), args=[Str(s='Test')], keywords=[]))
Expr(value=Call(func=Attribute(value=Name(id='ax'), attr='set_xlabel'), args=[Str(s='Temperature')], keywords=[keyword(arg='auth', value=Dict(keys=[Str(s='pwd')], values=[]))]))
Expr(value=Call(func=Attribute(value=Name(id='ax'), attr='set_xlabel'), args=[Str(s='Temperature')], keywords=[keyword(arg='auth', value=Dict(keys=[Str(s='pwd')], values=[]))]))
Expr(value=Call(func=Attribute(value=Name(id='json'), attr='dumps'), args=[Str(s='apple')], keywords=[]))
Expr(value=Call(func=Attribute(value=Str(s='\\xc3\\x85あ'), attr='format'), args=[Str(s='utf-8')], keywords=[]))
Expr(value=Call(func=Attribute(value=Name(id='ax'), attr='get'), args=[Str

Expr(value=Call(func=Attribute(value=Name(id='json'), attr='get'), args=[Str(s='http://httpbin.org/post')], keywords=[]))
Expr(value=Call(func=Attribute(value=Name(id='d'), attr='update'), args=[], keywords=[keyword(arg='key', value=Lambda(args=arguments(args=[arg(arg='d', annotation=None)], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=Subscript(value=Name(id='d'), slice=Index(value=Str(s='index')))))]))
Expr(value=Call(func=Name(id='dict'), args=[Call(func=Name(id='list'), args=[Call(func=Attribute(value=Name(id='d'), attr='items'), args=[], keywords=[])], keywords=[])], keywords=[]))
Expr(value=Call(func=Name(id='dict'), args=[Call(func=Name(id='list'), args=[Call(func=Attribute(value=Name(id='d'), attr='items'), args=[], keywords=[])], keywords=[])], keywords=[]))
Expr(value=Call(func=Name(id='dict'), args=[Call(func=Name(id='list'), args=[Call(func=Attribute(value=Name(id='d'), attr='items'), args=[], keywords=[])], keywords=[])], keywords=[]))
Expr(va

In [158]:
testFile.close()